### Import knihoven, parsování xml a vytvoření dataframe

In [115]:
import pandas as pd
import xml.etree.cElementTree as ET

tree = ET.parse("ISoSS_Opendata_EOSM.xml")
root = tree.getroot()

result = []
radek = []

for child in root:
        # nazev_obce
        radek.append(child[2][1].text)
        # nazev (místa)
        radek.append(child[3].text.strip().lower())
        # popis (místa)
        radek.append(child[4].text)
        # uvazek_vyse
        radek.append(child[5].text)
        # platova_trida
        radek.append(child[6].text)
        # prace_od
        radek.append(child[7].text)
        # min_vzdelani
        radek.append(child[8].text)
        # nabidka_platna_do
        radek.append(child[9].text)
        # 12 doba_urcita
        if child[12].tag.find("Doba") != -1:
            radek.append(child[12][1].text)
        else:
            radek.append("doba neurčitá")
        # 17 1 zadost_podani_organizace
        for elem in child.iter():
            if elem.tag.find("ZadostPodaniMisto") != -1:
                radek.append(elem[0].text)
        result.append(radek)
        radek = []

df = pd.DataFrame(data=result, columns=["nazev_obce", 
                                        "nazev", 
                                        "oddeleni_odbor", 
                                        "uvazek_vyse", 
                                        "platova_trida", 
                                        "prace_od", 
                                        "min_vzdelani", 
                                        "nabidka_platna_do",
                                        "doba_ne/urcita",
                                        "zadost_podani_organizace"
                                       ])
df.head()

,nazev_obce,nazev,oddeleni_odbor,uvazek_vyse,platova_trida,prace_od,min_vzdelani,nabidka_platna_do,doba_ne/urcita,zadost_podani_organizace
0,Praha,60422.13201 - vedoucí,Samostatné oddělení spisové a archivní s,1.0,14,2024-08-01,magisterský studijní program,2024-06-24,doba neurčitá,Ministerstvo dopravy
1,Praha,431.13401 - ministerský rada,"Oddělení metodik, koordinace",1.0,13,2024-08-01,magisterský studijní program,2024-07-03,nejdéle do 31. 12. 2026,Ministerstvo dopravy
2,Praha,dussm090 - rada,"Odd. dopravních, plynových, tlakových a",1.0,11,2024-08-01,bakalářský/magisterský studijní program,2024-06-30,doba neurčitá,Drážní úřad
3,Plzeň,dussm080 - odborný rada,Odd. pevných elektrických zařízení,1.0,11,2024-08-01,bakalářský/magisterský studijní program,2024-06-30,doba neurčitá,Drážní úřad
4,Praha,dussm013 - rada,Odd. vnitřní správy,1.0,10,2024-08-01,vyšší odb. vzdělání/bakalářský st. prog.,2024-06-30,doba neurčitá,Drážní úřad


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   nazev_obce                468 non-null    object
 1   nazev                     468 non-null    object
 2   oddeleni_odbor            468 non-null    object
 3   uvazek_vyse               468 non-null    object
 4   platova_trida             468 non-null    object
 5   prace_od                  468 non-null    object
 6   min_vzdelani              468 non-null    object
 7   nabidka_platna_do         468 non-null    object
 8   doba_ne/urcita            468 non-null    object
 9   zadost_podani_organizace  468 non-null    object
dtypes: object(10)
memory usage: 36.7+ KB


### Vyhledávání pomocí pozice

In [117]:
pozice = input("Pozice: ")
pozice = pozice.strip().lower()
df_final = df[df["nazev"].str.contains(pozice)].sort_values(by=["platova_trida"], ascending=False)
df_final

Pozice:  analyt


,nazev_obce,nazev,oddeleni_odbor,uvazek_vyse,platova_trida,prace_od,min_vzdelani,nabidka_platna_do,doba_ne/urcita,zadost_podani_organizace
39,Praha,000011500797 - finanční analytici,Oddělení zaklad. a zřiz. funkcí,1.0,13,2024-10-01,magisterský studijní program,2024-07-11,doba neurčitá,Ministerstvo obrany
142,Praha,5819591 - systémový analytik,"Oddělení bezpečnosti, krizového řízení a",1.0,13,2024-08-15,magisterský studijní program,2024-07-08,doba neurčitá,Správa státních hmotných rezerv
37,Praha,202300102611 - finanční analytici,Oddělení péče a podpory,1.0,12,2024-09-01,bakalářský/magisterský studijní program,2024-06-21,doba neurčitá,Ministerstvo obrany
76,Praha,32002022 - analytik velikosti podniku,Oddělení kontroly MSP,1.0,12,2024-08-01,magisterský studijní program,2024-06-24,po dobu výkonu veřejné funkce,Agentura pro podnikání a inovace
93,Praha,ssm217 - analytik/analytička,Oddělení analytické chemie - ANA,1.0,12,2024-07-01,bakalářský/magisterský studijní program,2024-06-21,doba neurčitá,Státní ústav pro kontrolu léčiv
